In [9]:
import time
import datetime
import requests
import pandas as pd
import os

In [10]:

def fetch_eia_data(api_path, *,api_key=None, frequency=None, facets=None,
                   data_vars=None, start=None, end=None,
                   offset=0, length=5000):
    """Fetch one “page” of data from EIA v2."""
    base = "https://api.eia.gov/v2/"
    url = f"{base}{api_path.rstrip('/')}/data/"
    
    # Build params according to EIA spec
    params = {
        "api_key": api_key,
        "offset": offset,
        "length": length
    }
    if frequency is not None:
        params["frequency"] = frequency
    if data_vars is not None:
        # EIA expects something like data[0]=value, data[1]=other
        for i, dv in enumerate(data_vars):
            params[f"data[{i}]"] = dv
    if facets is not None:
        for facet_key, facet_vals in facets.items():
            for i, val in enumerate(facet_vals):
                params[f"facets[{facet_key}][{i}]"] = val
    if start is not None:
        params["start"] = start
    if end is not None:
        params["end"] = end
    
    print("Requesting:", url)
    print("Params:", {p: params[p] for p in params if p!= 'api_key'})
    
    resp = requests.get(url, params=params)
    try:
        resp.raise_for_status()
    except requests.HTTPError as e:
        print("ERROR response status:", resp.status_code)
        print("Response text:", resp.text)
        raise
    
    return resp.json()




In [11]:
def fetch_all_for_series(api_path, **kwargs):
    """Fetch all pages (offset-based) for a series, merging them.

    Returns: 
            pandas.DataFrame: a dataframe containing all rows from the api querey
    """
    all_records = []
    offset = 0
    length = kwargs.get("length")
    
    while True:
        response_json = fetch_eia_data(api_path, offset=offset, **kwargs)
        recs = response_json["response"]["data"]
        if not recs:
            break
        all_records.extend(recs)
        # If fewer than length returned, we've reached the end so we're done'
        if len(recs) < length:
            break
        offset += length
        # sleep to slow requests
        time.sleep(0.2)

    #df 
    
    return pd.DataFrame(all_records)

In [ ]:
def get_api_key():
    return "put your api key here"

## Electricity Sales price

In [13]:
api_path = "electricity/retail-sales"

# use this to filter by state or sector e.g.
# facets = {"stateid: ['CA', 'WI'],
#           "sectorid": ['IND', 'RES'] 
#           }
# to get only date for industrial and residential sectors in california and wisconsin
facets=None

# remember to remove api key before committing to repo
API_KEY = get_api_key()

kwargs = {
    "api_key": API_KEY,
    "frequency": "monthly",
    "data_vars": ["customers", "price", "revenue", "sales"],  # actual data variables
    "facets": facets,
    "start": "2001-01", #start date - Jan 2001 is as far back as we can go
    "end": "2025-07", #end date
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/electricity/retail-sales?frequency=monthly&data=customers;price;revenue;sales;&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)

# drop the columns for stateDescription and sectorName since they are redundant
df = df.drop(columns=['stateDescription', 'sectorName'])

Requesting: https://api.eia.gov/v2/electricity/retail-sales/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'monthly', 'data[0]': 'customers', 'data[1]': 'price', 'data[2]': 'revenue', 'data[3]': 'sales', 'start': '2001-01', 'end': '2025-07'}
Requesting: https://api.eia.gov/v2/electricity/retail-sales/data/
Params: {'offset': 5000, 'length': 5000, 'frequency': 'monthly', 'data[0]': 'customers', 'data[1]': 'price', 'data[2]': 'revenue', 'data[3]': 'sales', 'start': '2001-01', 'end': '2025-07'}
Requesting: https://api.eia.gov/v2/electricity/retail-sales/data/
Params: {'offset': 10000, 'length': 5000, 'frequency': 'monthly', 'data[0]': 'customers', 'data[1]': 'price', 'data[2]': 'revenue', 'data[3]': 'sales', 'start': '2001-01', 'end': '2025-07'}
Requesting: https://api.eia.gov/v2/electricity/retail-sales/data/
Params: {'offset': 15000, 'length': 5000, 'frequency': 'monthly', 'data[0]': 'customers', 'data[1]': 'price', 'data[2]': 'revenue', 'data[3]': 'sales', 'start': '2001-01',

In [14]:
import os
data_dir = os.path.join(os.pardir, 'datasets')

df.to_csv(os.path.join(data_dir, 'state_electricity_price.csv'), index=False)

# multi-index the dataframe first by month and year then by state

df = df.set_index(["period", "stateid", "sectorid"]).sort_index()

df
#df.to_csv('state_electricity_price.csv')


customers  price   revenue      sales  \
period  stateid sectorid                                         
2001-01 AK      ALL           None   9.97  51.96404  521.03566   
                COM           None    9.7   20.2141  208.49968   
                IND           None   7.05   6.26039    88.7627   
                OTH           None  13.36   2.36091   17.66485   
                RES           None  11.22  23.12865  206.10843   
...                            ...    ...       ...        ...   
2025-07 WY      COM          60181  10.16  46.39331  456.52285   
                IND          12067   8.93  68.35194  765.72477   
                OTH           None   None      None       None   
                RES         289795  14.64  34.96128   238.7893   
                TRA              0      0         0          0   

                              customers-units              price-units  \
period  stateid sectorid                                                 
2001-01 AK      ALL       number of customers  cents per kilowatt-hour   
                COM       number of customers  cents per kilowatt-hour   
                IND       number of customers  cents per kilowatt-hour   
                OTH       number of customers  cents per kilowatt-hour   
                RES       number of customers  cents per kilowatt-hour   
...                                       ...                      ...   
2025-07 WY      COM       number of customers  cents per kilowatt-hour   
                IND       number of customers  cents per kilowatt-hour   
                OTH       number of customers  cents per kilowatt-hour   
                RES       number of customers  cents per kilowatt-hour   
                TRA       number of customers  cents per kilowatt-hour   

                            revenue-units             sales-units  
period  stateid sectorid                                           
2001-01 AK      ALL       million dollars  million kilowatt hours  
                COM       million dollars  million kilowatt hours  
                IND       million dollars  million kilowatt hours  
                OTH       million dollars  million kilowatt hours  
                RES       million dollars  million kilowatt hours  
...                                   ...                     ...  
2025-07 WY      COM       million dollars  million kilowatt hours  
                IND       million dollars  million kilowatt hours  
                OTH       million dollars  million kilowatt hours  
                RES       million dollars  million kilowatt hours  
                TRA       million dollars  million kilowatt hours  

[109740 rows x 8 columns]

## Electricity Generation

In [15]:
api_path = "electricity/electric-power-operational-data"

states= [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

# sectorid 99 is all sectors
facets= {'sectorid': ['99'], 'location': states} 

# remember to remove api key before committing to repo
API_KEY = get_api_key()

kwargs = {
    "api_key": API_KEY,
    "frequency": "monthly",
    "data_vars": ["generation"],  # actual data variables
    "facets": facets,
    "start": "2001-01", #start date - Jan 2001 is as far back as we can go
    "end": "2025-07", #end date
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/electricity/retail-sales?frequency=monthly&data=customers;price;revenue;sales;&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)

# drop the columns for stateDescription
df = df.drop(columns=['stateDescription'])

#rename location column to stateid to match electricity price data
df = df.rename(columns={'location': 'stateid'})

print(datetime.datetime.now())


Requesting: https://api.eia.gov/v2/electricity/electric-power-operational-data/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'monthly', 'data[0]': 'generation', 'facets[sectorid][0]': '99', 'facets[location][0]': 'AL', 'facets[location][1]': 'AK', 'facets[location][2]': 'AZ', 'facets[location][3]': 'AR', 'facets[location][4]': 'CA', 'facets[location][5]': 'CO', 'facets[location][6]': 'CT', 'facets[location][7]': 'DE', 'facets[location][8]': 'FL', 'facets[location][9]': 'GA', 'facets[location][10]': 'HI', 'facets[location][11]': 'ID', 'facets[location][12]': 'IL', 'facets[location][13]': 'IN', 'facets[location][14]': 'IA', 'facets[location][15]': 'KS', 'facets[location][16]': 'KY', 'facets[location][17]': 'LA', 'facets[location][18]': 'ME', 'facets[location][19]': 'MD', 'facets[location][20]': 'MA', 'facets[location][21]': 'MI', 'facets[location][22]': 'MN', 'facets[location][23]': 'MS', 'facets[location][24]': 'MO', 'facets[location][25]': 'MT', 'facets[location][26]': 'NE',

In [16]:
data_dir = os.path.join(os.pardir, 'datasets')
df.to_csv(os.path.join(data_dir, 'state_electricity_generation.csv'), index=False)

df
#df.to_csv('state_electricity_price.csv')

,period,stateid,sectorid,sectorDescription,fueltypeid,fuelTypeDescription,generation,generation-units
0,2009-01,NJ,99,All Sectors,ALL,all fuels,5675.1666,thousand megawatthours
1,2009-01,NJ,99,All Sectors,BIO,biomass,67.6432,thousand megawatthours
2,2009-01,NJ,99,All Sectors,BIS,bituminous coal and synthetic coal,654.45299,thousand megawatthours
3,2009-01,NJ,99,All Sectors,COL,"coal, excluding waste coal",818.22594,thousand megawatthours
4,2009-01,NJ,99,All Sectors,COW,all coal products,818.22594,thousand megawatthours
...,...,...,...,...,...,...,...,...
440873,2009-07,AK,99,All Sectors,ALL,all fuels,505.30617,thousand megawatthours
440874,2009-07,AK,99,All Sectors,BIO,biomass,.57707,thousand megawatthours
440875,2009-07,AK,99,All Sectors,COW,all coal products,53.79728,thousand megawatthours
440876,2009-07,AK,99,All Sectors,FOS,fossil fuels,400.08031,thousand megawatthours


## Other annual state specific data

In [17]:
api_path = "electricity/state-electricity-profiles/source-disposition"

states= [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

facets= None
data_vars = [
        "facility-direct",
        "net-interstate-trade",
        "total-elect-indust",
        "total-international-exports",
        "total-international-imports",
        "total-net-generation"
    ]

# remember to remove api key before committing to repo
API_KEY = get_api_key()

kwargs = {
    "api_key": API_KEY,
    "frequency": "annual", #only annual data available from this db
    "data_vars": data_vars,  # actual data variables
    "facets": facets,
    "start": "2001", #start date - Jan 2001 is as far back as we can go
    "end": "2023", #end date- 2023 is as far as we can go
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/electricity/state-electricity-profiles/source-disposition?frequency=annual&data=facility-direct;net-interstate-trade;total-elect-indust;total-international-exports;total-international-imports;total-net-generation;&start=2001&end=2023&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)

# drop the columns for stateDescription
df = df.drop(columns=['stateDescription'])

#rename state column to stateid to match electricity price data
df = df.rename(columns={'state': 'stateid'})

print(datetime.datetime.now())

Requesting: https://api.eia.gov/v2/electricity/state-electricity-profiles/source-disposition/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'annual', 'data[0]': 'facility-direct', 'data[1]': 'net-interstate-trade', 'data[2]': 'total-elect-indust', 'data[3]': 'total-international-exports', 'data[4]': 'total-international-imports', 'data[5]': 'total-net-generation', 'start': '2001', 'end': '2023'}
2025-09-29 22:32:06.236497


In [18]:
data_dir = os.path.join(os.pardir, 'datasets')
df.to_csv(os.path.join(data_dir, 'state_electricity_annual_data.csv'), index=False)

df
#df.to_csv('state_electricity_price.csv')

,period,stateid,facility-direct,net-interstate-trade,total-elect-indust,total-international-exports,total-international-imports,total-net-generation,facility-direct-units,net-interstate-trade-units,total-elect-indust-units,total-international-exports-units,total-international-imports-units,total-net-generation-units
0,2013,MN,942126,-15273728,68644103,262439,8179786,51296988,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
1,2017,MO,None,3165496,76461419,0,0,84606731,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
2,2015,MO,None,-3468206,81504081,0,0,83640067,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
3,2004,MO,19684,4488629,74054296,6276,0,87632910,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
4,2006,MO,19690,433931,82015230,10,2975,91686343,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1191,2022,CO,179780,-1909941,56763041,0,0,58044009,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
1192,2002,CO,141066,-5681503,46077762,0,6707,45600388,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
1193,2023,CO,175842,-1018485,55565819,0,0,57541720,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours
1194,2007,CO,248794,-1726737,51299156,1679,1268,53907492,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours,megawatthours


## Coal

### Export and Import prices

In [19]:
api_path = "coal/exports-imports-quantity-price"

states= [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

facets= {'countryId': ['TOT'], 'customsDistrictId': ['TOT']}
data_vars = ['price']

# remember to remove api key before committing to repo
API_KEY = get_api_key()

kwargs = {
    "api_key": API_KEY,
    "frequency": "quarterly", #only quarterly data available from this db
    "data_vars": data_vars,  # actual data variables
    "facets": facets,
    "start": "2001-Q1", #start date
    "end": "2024-Q1", #end date
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/coal/exports-imports-quantity-price?frequency=quarterly&data=price;&facets=countryId;customsDistrictId;&countryId=TOT;&customsDistrictId=TOT;&start=2001-01&end=2024-01&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)

# drop theses columns because we only look at total imports (not broken down by exporting country and location of import)
df = df.drop(columns=['countryId', 'countryDescription', 'customsDistrictId', 'customsDistrictDescription'])

print(datetime.datetime.now())

Requesting: https://api.eia.gov/v2/coal/exports-imports-quantity-price/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'quarterly', 'data[0]': 'price', 'facets[countryId][0]': 'TOT', 'facets[customsDistrictId][0]': 'TOT', 'start': '2001-Q1', 'end': '2024-Q1'}
2025-09-29 22:32:07.160640


In [20]:
data_dir = os.path.join(os.pardir, 'datasets')
df.to_csv(os.path.join(data_dir, 'quarterly_coal_import_price.csv'), index=False)

df
#df.to_csv('state_electricity_price.csv')

,period,exportImportType,coalRankId,coalRankDescription,price,price-units
0,2008-Q1,Exports,MET,Metallurgical,98.9,dollars per short ton
1,2010-Q3,Imports,MET,Metallurgical,204.71,dollars per short ton
2,2004-Q2,Exports,MET,Metallurgical,65.32,dollars per short ton
3,2004-Q1,Exports,MET,Metallurgical,54.1,dollars per short ton
4,2004-Q1,Imports,MET,Metallurgical,48.42,dollars per short ton
...,...,...,...,...,...,...
739,2023-Q1,Imports,STM,Steam Coal,129.34,dollars per short ton
740,2023-Q3,Imports,STM,Steam Coal,114.71,dollars per short ton
741,2023-Q1,Exports,STM,Steam Coal,119.62,dollars per short ton
742,2023-Q3,Exports,STM,Steam Coal,85.74,dollars per short ton


### Coal consumption and prices

In [26]:
api_path = "coal/consumption-and-quality"

states= [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

facets= {'location': states}
data_vars = ['price', 'consumption', 'stocks']

# remember to remove api key before committing to repo
API_KEY = get_api_key()

kwargs = {
    "api_key": API_KEY,
    "frequency": "quarterly", #only quarterly data available from this db
    "data_vars": data_vars,  # actual data variables
    "facets": facets,
    "start": "2001-Q1", #start date
    "end": "2024-Q1", #end date
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/coal/consumption-and-quality?frequency=quarterly&data=consumption;price;stocks;&facets=location;&location=AR;&start=2001-01&end=2024-01&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)

# drop theses columns because we only look at total imports (not broken down by exporting country and location of import)
df = df.drop(columns=['stateDescription'])

#rename state column to stateid to match electricity price data
df = df.rename(columns={'state': 'stateid'})

print(datetime.datetime.now())

Requesting: https://api.eia.gov/v2/coal/consumption-and-quality/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'quarterly', 'data[0]': 'price', 'data[1]': 'consumption', 'data[2]': 'stocks', 'facets[location][0]': 'AL', 'facets[location][1]': 'AK', 'facets[location][2]': 'AZ', 'facets[location][3]': 'AR', 'facets[location][4]': 'CA', 'facets[location][5]': 'CO', 'facets[location][6]': 'CT', 'facets[location][7]': 'DE', 'facets[location][8]': 'FL', 'facets[location][9]': 'GA', 'facets[location][10]': 'HI', 'facets[location][11]': 'ID', 'facets[location][12]': 'IL', 'facets[location][13]': 'IN', 'facets[location][14]': 'IA', 'facets[location][15]': 'KS', 'facets[location][16]': 'KY', 'facets[location][17]': 'LA', 'facets[location][18]': 'ME', 'facets[location][19]': 'MD', 'facets[location][20]': 'MA', 'facets[location][21]': 'MI', 'facets[location][22]': 'MN', 'facets[location][23]': 'MS', 'facets[location][24]': 'MO', 'facets[location][25]': 'MT', 'facets[location][26]': 'NE',

In [27]:
data_dir = os.path.join(os.pardir, 'datasets')
df.to_csv(os.path.join(data_dir, 'quarterly_coal_price_consumption.csv'), index=False)

df
#df.to_csv('state_electricity_price.csv')

,period,location,sector,sectorDescription,price,consumption,stocks,price-units,consumption-units,stocks-units
0,2001-Q1,AK,1,Electric Utility,None,43623,0,dollars per short ton,short tons,short tons
1,2001-Q1,AL,1,Electric Utility,31.18,8299678,2908451,dollars per short ton,short tons,short tons
2,2001-Q1,AR,1,Electric Utility,19.74,3165885,2020072,dollars per short ton,short tons,short tons
3,2001-Q1,AZ,1,Electric Utility,25.74,4667401,1876102,dollars per short ton,short tons,short tons
4,2001-Q1,CO,1,Electric Utility,17.91,4831350,2766003,dollars per short ton,short tons,short tons
...,...,...,...,...,...,...,...,...,...,...
22434,2024-Q1,VA,98,Electric Power,103.72,272765,None,dollars per short ton,short tons,short tons
22435,2024-Q1,WA,98,Electric Power,None,556178,None,dollars per short ton,short tons,short tons
22436,2024-Q1,WI,98,Electric Power,40.93,2968338,None,dollars per short ton,short tons,short tons
22437,2024-Q1,WV,98,Electric Power,None,4479269,None,dollars per short ton,short tons,short tons


## natural gas futures prices

In [21]:
api_path = "natural-gas/pri/fut"


facets= None
data_vars = ['value']

# remember to remove api key before committing to repo
API_KEY = get_api_key()

kwargs = {
    "api_key": API_KEY,
    "frequency": "monthly", 
    "data_vars": data_vars,  # actual data variables
    "facets": facets,
    "start": "2001-01", #start date
    "end": "2025-07", #end date
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/natural-gas/pri/fut?frequency=monthly&data=value;&start=2001-01&end=2025-07&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)


print(datetime.datetime.now())

Requesting: https://api.eia.gov/v2/natural-gas/pri/fut/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'monthly', 'data[0]': 'value', 'start': '2001-01', 'end': '2025-07'}
2025-09-29 22:32:09.836327


In [22]:
data_dir = os.path.join(os.pardir, 'datasets')
df.to_csv(os.path.join(data_dir, 'nat_gas_futures.csv'), index=False)

df
#df.to_csv('state_electricity_price.csv')

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2001-10,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE1,Future Contract 1,RNGC1,Natural Gas Futures Contract 1 (Dollars per Mi...,2.618,$/MMBTU
1,2001-11,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE1,Future Contract 1,RNGC1,Natural Gas Futures Contract 1 (Dollars per Mi...,2.786,$/MMBTU
2,2003-09,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE1,Future Contract 1,RNGC1,Natural Gas Futures Contract 1 (Dollars per Mi...,4.667,$/MMBTU
3,2003-12,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE1,Future Contract 1,RNGC1,Natural Gas Futures Contract 1 (Dollars per Mi...,6.469,$/MMBTU
4,2004-05,Y35NY,NEW YORK CITY,EPG0,Natural Gas,PE1,Future Contract 1,RNGC1,Natural Gas Futures Contract 1 (Dollars per Mi...,6.398,$/MMBTU
...,...,...,...,...,...,...,...,...,...,...,...
1607,2019-10,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,2.33,$/MMBTU
1608,2020-06,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,1.63,$/MMBTU
1609,2022-05,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,8.14,$/MMBTU
1610,2024-08,RGC,NA,EPG0,Natural Gas,PS0,Spot Price,RNGWHHD,Henry Hub Natural Gas Spot Price (Dollars per ...,1.99,$/MMBTU


## Crude Oil and Propane Futures

In [24]:
api_path = "petroleum/pri/fut"


facets= {'product': ['EPC0', 'EPLLPA']}
data_vars = ['value']

# remember to remove api key before committing to repo
API_KEY = get_api_key()

kwargs = {
    "api_key": API_KEY,
    "frequency": "monthly", 
    "data_vars": data_vars,  # actual data variables
    "facets": facets,
    "start": "2001-01", #start date
    "end": "2024-04", #end date
    "length": 5000 #rows of data per query
}


# queries the data found here: https://www.eia.gov/opendata/browser/petroleum/pri/fut?frequency=monthly&data=value;&facets=product;&product=EPC0;EPLLPA;&sortColumn=period;&sortDirection=desc;
df = fetch_all_for_series(api_path, **kwargs)


print(datetime.datetime.now())

Requesting: https://api.eia.gov/v2/petroleum/pri/fut/data/
Params: {'offset': 0, 'length': 5000, 'frequency': 'monthly', 'data[0]': 'value', 'facets[product][0]': 'EPC0', 'facets[product][1]': 'EPLLPA', 'start': '2001-01', 'end': '2024-04'}
2025-09-29 22:40:41.778013


In [25]:
data_dir = os.path.join(os.pardir, 'datasets')
df.to_csv(os.path.join(data_dir, 'oil_futures.csv'), index=False)

df
#df.to_csv('state_electricity_price.csv')

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2001-05,Y44MB,NA,EPLLPA,Propane,PE1,Future Contract 1,EER_EPLLPA_PE1_Y44MB_DPG,"Mont Belvieu, Tx Propane Future Contract 1 (Do...",.516,$/GAL
1,2001-06,Y44MB,NA,EPLLPA,Propane,PE1,Future Contract 1,EER_EPLLPA_PE1_Y44MB_DPG,"Mont Belvieu, Tx Propane Future Contract 1 (Do...",.437,$/GAL
2,2003-03,Y44MB,NA,EPLLPA,Propane,PE1,Future Contract 1,EER_EPLLPA_PE1_Y44MB_DPG,"Mont Belvieu, Tx Propane Future Contract 1 (Do...",.568,$/GAL
3,2004-04,Y44MB,NA,EPLLPA,Propane,PE1,Future Contract 1,EER_EPLLPA_PE1_Y44MB_DPG,"Mont Belvieu, Tx Propane Future Contract 1 (Do...",.607,$/GAL
4,2004-06,Y44MB,NA,EPLLPA,Propane,PE1,Future Contract 1,EER_EPLLPA_PE1_Y44MB_DPG,"Mont Belvieu, Tx Propane Future Contract 1 (Do...",.671,$/GAL
...,...,...,...,...,...,...,...,...,...,...,...
415,2007-05,Y44MB,NA,EPLLPA,Propane,PE4,Future Contract 4,EER_EPLLPA_PE4_Y44MB_DPG,"Mont Belvieu, Tx Propane Future Contract 4 (Do...",1.154,$/GAL
416,2007-06,Y44MB,NA,EPLLPA,Propane,PE4,Future Contract 4,EER_EPLLPA_PE4_Y44MB_DPG,"Mont Belvieu, Tx Propane Future Contract 4 (Do...",1.155,$/GAL
417,2007-07,Y44MB,NA,EPLLPA,Propane,PE4,Future Contract 4,EER_EPLLPA_PE4_Y44MB_DPG,"Mont Belvieu, Tx Propane Future Contract 4 (Do...",1.209,$/GAL
418,2009-01,Y44MB,NA,EPLLPA,Propane,PE4,Future Contract 4,EER_EPLLPA_PE4_Y44MB_DPG,"Mont Belvieu, Tx Propane Future Contract 4 (Do...",.708,$/GAL
